In [17]:
import mujoco_py
import os
import numpy as np
from mujoco_dynamics import MujocoDynamics
import multiprocessing as mp
import time

In [2]:
xml_path = os.path.join(os.getcwd(), 'xmls', 'inverted_pendulum.xml')
model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)

In [4]:
sim.data.qpos[:] = [0.00184113, -0.00434254]
sim.data.qvel[:] = [0.18378366, -0.43146432]
print(sim.data.qpos, sim.data.qvel)
sim.data.ctrl[:] = [2.28478247]
sim.step()
print(sim.data.qpos, sim.data.qvel)

[ 0.00184113 -0.00434254] [ 0.18378366 -0.43146432]
[ 0.00970685 -0.0227588 ] [ 0.60210889 -1.40465707]


In [5]:
print(sim.data.qpos, sim.data.qvel)
sim.data.ctrl[:] = [1.000001]
sim.step()
print(sim.data.qpos, sim.data.qvel)

[ 0.00970685 -0.0227588 ] [ 0.60210889 -1.40465707]
[ 0.02353548 -0.05476015] [ 0.78066916 -1.79556659]


In [6]:
print(sim.model.opt.timestep)
print(sim.model.actuator_ctrlrange)

0.02
[[-3.  3.]]


In [7]:
print(np.concatenate([sim.data.qpos, sim.data.qvel]))
print(sim.model.nq, sim.model.nv)

[ 0.02353548 -0.05476015  0.78066916 -1.79556659]
2 2


In [8]:
dynamics = MujocoDynamics(xml_path)

In [9]:
print(dynamics.get_state())
dynamics.set_state(np.array([0.00184113, -0.00434254, 0.18378366, -0.43146432]))
print(dynamics.step(np.array([1.0])), dynamics.constrain(np.array([1.0])))

[0. 0. 0. 0.]
[ 0.00970685 -0.0227588   0.60210889 -1.40465707] [2.28478247]


In [10]:
viewer = mujoco_py.MjViewer(sim)
for i in range(1000):
    sim.step()
    viewer.render()

Creating window glfw


SystemExit: 0

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
sim.data.qpos[0] += 1
print(sim.data.qpos, sim.data.qvel)

[ 4.00086607 -9.7618978 ] [ 5.91281464e-05 -8.08604038e-01]


In [12]:
for i in range(2000):
    sim.step()
print(sim.data.qpos, sim.data.qvel)

[  3.00080341 -21.99281524] [-4.10269552e-15 -6.74079068e-13]


In [13]:
from mujoco_dynamics import MujocoDynamics
import os
import numpy as np
import multiprocessing as mp

In [19]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'inverted_pendulum.xml')
dynamics = MujocoDynamics(xml_path, frame_skip = 2)

In [92]:
def _worker_init(xml_path):
    global sim
    model = mujoco_py.load_model_from_path(xml_path)
    sim = mujoco_py.MjSim(model)
    print("hi")
def _worker(u):
    sim.reset()
    for i in range(100):
        sim.step(u)
    
pool = mp.Pool(initializer = _worker_init, initargs = (xml_path,))



hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


In [93]:
t0 = time.time()
results = pool.map(_worker, [np.array([0.0]) for i in range(100)])
print(time.time() - t0)

0.0238189697265625


In [94]:
pool.close()

In [95]:
t0 = time.time()

model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)
for i in range(100):
    sim.reset()
    for i in range(100):
        sim.step(np.array([0.0]))

print(time.time() - t0)

0.08756303787231445


In [100]:
t = ([1,2], [3,4], [5,6])
tuple(zip(*t))

((1, 3, 5), (2, 4, 6))

In [1]:
import mujoco_py_deriv

ModuleNotFoundError: No module named 'mujoco_py_deriv'